In [137]:
from typing import Tuple, Union

import chess
import numpy as np
from chess import engine, pgn
from tensorflow import data

In [146]:
def fen_to_bitmap(fen: str) -> np.ndarray:
    board = chess.BaseBoard(fen)
    bitmap = np.zeros((len(chess.FILE_NAMES), len(chess.RANK_NAMES), len(chess.PIECE_TYPES)))
    for piece_idx, piece_type in enumerate(chess.PIECE_TYPES):
        for file_idx in range(len(chess.FILE_NAMES)):
            for rank_idx in range(len(chess.RANK_NAMES)):
                piece = board.piece_at(file_idx * len(chess.RANK_NAMES) + rank_idx)
                if piece and piece.piece_type is piece_type:
                    bitmap[file_idx, rank_idx, piece_idx] = 1 if piece.color is chess.WHITE else -1
    return bitmap

In [166]:
def cp_to_int(info: dict) -> int:
    # return info["score"]
    return 1

In [167]:
def generate(file: Union[bytes, str], command: str) -> Tuple[str, int]:

    # BUG: Tensorflow converts input to bytes
    command = command.decode("utf-8")

    simple_engine = engine.SimpleEngine.popen_uci(command)
    with open(file) as f:
        while True:
            game = pgn.read_game(f)
            if not game:
                break
            mainline = game.mainline()
            if mainline:
                board = np.random.choice(list(mainline)).board()
                bitmap = fen_to_bitmap(board.board_fen())
                score = cp_to_int(simple_engine.analyse(board, engine.Limit()))
                yield bitmap, score

In [164]:
dataset = data.Dataset.from_generator(
    generate, (np.int8, np.int8),
    output_shapes=((8, 8, 6), ()),
    args=(
        "../../data/2019.pgn",
        "../../lib/stockfish-11-win/Windows/stockfish_20011801_x64_modern.exe"
    )
)

In [168]:
dataset

<FlatMapDataset shapes: ((8, 8, 6), ()), types: (tf.int8, tf.int8)>